In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import joblib
import os
os.chdir('Resources/')

In [2]:
df = pd.read_csv('7_CC_Encrypted_Data.csv')
df = df.dropna()

In [3]:
Y = df[['HeartDisease']]
Y

,HeartDisease
0,0
1,1
2,0
3,1
4,0
...,...
1817,0
1818,0
1819,0
1820,0


In [4]:
c1_list = []
c2_list = []

for k in range(11):
    cnt = 0
    c1_sum = 0
    c2_sum = 0
    
    messages = df.iloc[:, k].values

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))
            c1_sum += c1
            c2_sum += c2
            cnt += 1
    
    print(c1_sum//cnt, ",", c2_sum//cnt)

    c1_list.append(c1_sum//cnt)
    c2_list.append(c2_sum//cnt)

print(c1_list)
print(c2_list)

7270988666945721113120809463312144500555280693674 , 32083111036126459261180639309633731096371489192527
22961665442842143903220898566890700943449852505536 , 50806696935996374692315166258825721820101871429809
20613262322062277119573630365746595127182499134519 , 22816956567510665168741835293166565473220634279942
25666449378145533821257044298909866450131224188388 , 43308947551399742961005592210604413184580574131025
49648777350264920336257917599386766751429992537949 , 18080880950383506422733101513743633287139900422079
23262138535430815965897209751063602705729030703607 , 36371123739210616465996057708704981096130718166775
20663312364214574545779485666796513311973587596483 , 43229146032083899182761576784977586741533298306210
52792865455644029778549137754509322291183002304751 , 29013720512811045293191661764715330706360614478768
53567916807806914103914877537850847808169443022861 , 21498702084474512676857271979017679975982111913956
22453991272873058234492300158377019032521111612219 , 208439164278

In [5]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

In [6]:
imputed_data = pd.DataFrame()

for k in range(11):
    
    messages = df.iloc[:, k].values

    imputed_column = []

    for i, message in enumerate(messages, 1):
        if message is None:
            c1 = c1_list[k]
            c2 = c2_list[k]
        
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))

        imputed_column.append(f"({c1},{c2})")

    imputed_data[column_names[k]] = imputed_column

In [7]:
scaler = joblib.load('13_CC_Scaler.joblib')

In [8]:
df_scaled = scaler.transform(imputed_data)
df_scaled = pd.DataFrame(df_scaled, columns=imputed_data.columns)
df_scaled.to_csv('14_CC_Scaled_Data.csv', index=False)

g:\Thesis - Paper - Proper\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [9]:
import ast

random_state = None

with open("13_CC_Hyperparameters.txt", "r") as f:
    for line in f:
        if line.startswith("train_test_split"):
            param_str = line.split("Hyperparameters: ")[1].strip()
            params = ast.literal_eval(param_str)
            random_state = params.get("random_state")

print("Random State for Train-Test Split:", random_state)

Random State for Train-Test Split: 2419


In [14]:
# Best Model validation
#-----------------------

rf = joblib.load('12_SP_Model_RF.joblib')

file_path = "13_CC_Matrices_RF.txt"

with open(file_path, "r") as f:
    lines = f.read().strip().splitlines()
    reference_f1 = float(lines[0])
    reference_acc = float(lines[1])
    reference_pre = float(lines[2])
    reference_rec = float(lines[3])

print("With Same Test Data: ")
print(f"--------------------------------------------------------------------------------------------------------------------------")
print(f"Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference")
print(f"F1-score \t F1-score \t | Accuracy \t Accuracy \t | Precision \t Precision \t | Recall \t Recall")
print(f"--------------------------------------------------------------------------------------------------------------------------")
X_train, X_test, Y_train, Y_test = train_test_split(df_scaled, Y, test_size=0.2, random_state=random_state)
rf_pred = rf.predict(X_test)
calculated_f1 = f1_score(Y_test, rf_pred)
calculated_acc = accuracy_score(Y_test, rf_pred)
calculated_pre = precision_score(Y_test, rf_pred)
calculated_rec = recall_score(Y_test, rf_pred)
print(f"{calculated_f1 * 100 : .2f}%, \t {reference_f1 * 100 : .2f}% \t | {calculated_acc * 100 : .2f}%, \t {reference_acc * 100 : .2f}% \t | {calculated_pre * 100 : .2f}%, \t {reference_pre * 100 : .2f}% \t | {calculated_rec * 100 : .2f}%, \t {reference_rec * 100 : .2f}%")

print("\n\nWith Different Test Data: ")
print(f"--------------------------------------------------------------------------------------------------------------------------")
print(f"Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference")
print(f"F1-score \t F1-score \t | Accuracy \t Accuracy \t | Precision \t Precision \t | Recall \t Recall")
print(f"--------------------------------------------------------------------------------------------------------------------------")
for i in range(15):
    X_train, X_test, Y_train, Y_test = train_test_split(df_scaled, Y, test_size=0.9)
    rf_pred = rf.predict(X_test)
    calculated_f1 = f1_score(Y_test, rf_pred)
    calculated_acc = accuracy_score(Y_test, rf_pred)
    calculated_pre = precision_score(Y_test, rf_pred)
    calculated_rec = recall_score(Y_test, rf_pred)

    print(f"{calculated_f1 * 100 : .2f}%, \t {reference_f1 * 100 : .2f}% \t | {calculated_acc * 100 : .2f}%, \t {reference_acc * 100 : .2f}% \t | {calculated_pre * 100 : .2f}%, \t {reference_pre * 100 : .2f}% \t | {calculated_rec * 100 : .2f}%, \t {reference_rec * 100 : .2f}%")

With Same Test Data: 
--------------------------------------------------------------------------------------------------------------------------
Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference
F1-score 	 F1-score 	 | Accuracy 	 Accuracy 	 | Precision 	 Precision 	 | Recall 	 Recall
--------------------------------------------------------------------------------------------------------------------------
 92.86%, 	  92.86% 	 |  91.23%, 	  91.23% 	 |  91.63%, 	  91.63% 	 |  94.12%, 	  94.12%


With Different Test Data: 
--------------------------------------------------------------------------------------------------------------------------
Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference
F1-score 	 F1-score 	 | Accuracy 	 Accuracy 	 | Precision 	 Precision 	 | Recall 	 Recall
--------------------------------------------------------------------------------------------------------

In [16]:
# Fake Model validation
#-----------------------

rf = joblib.load('12_SP_Fake_Model_RF.joblib')

file_path = "13_CC_Fake_Matrices_RF.txt"

with open(file_path, "r") as f:
    lines = f.read().strip().splitlines()
    reference_f1 = float(lines[0])
    reference_acc = float(lines[1])
    reference_pre = float(lines[2])
    reference_rec = float(lines[3])

print("With Same Test Data: ")
print(f"--------------------------------------------------------------------------------------------------------------------------")
print(f"Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference")
print(f"F1-score \t F1-score \t | Accuracy \t Accuracy \t | Precision \t Precision \t | Recall \t Recall")
print(f"--------------------------------------------------------------------------------------------------------------------------")
X_train, X_test, Y_train, Y_test = train_test_split(df_scaled, Y, test_size=0.2, random_state=random_state)
rf_pred = rf.predict(X_test)
calculated_f1 = f1_score(Y_test, rf_pred)
calculated_acc = accuracy_score(Y_test, rf_pred)
calculated_pre = precision_score(Y_test, rf_pred)
calculated_rec = recall_score(Y_test, rf_pred)
print(f"{calculated_f1 * 100 : .2f}%, \t {reference_f1 * 100 : .2f}% \t | {calculated_acc * 100 : .2f}%, \t {reference_acc * 100 : .2f}% \t | {calculated_pre * 100 : .2f}%, \t {reference_pre * 100 : .2f}% \t | {calculated_rec * 100 : .2f}%, \t {reference_rec * 100 : .2f}%")

print("\n\nWith Different Test Data: ")
print(f"--------------------------------------------------------------------------------------------------------------------------")
print(f"Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference \t | Calculated \t Reference")
print(f"F1-score \t F1-score \t | Accuracy \t Accuracy \t | Precision \t Precision \t | Recall \t Recall")
print(f"--------------------------------------------------------------------------------------------------------------------------")
for i in range(15):
    X_train, X_test, Y_train, Y_test = train_test_split(df_scaled, Y, test_size=0.9)
    rf_pred = rf.predict(X_test)
    calculated_f1 = f1_score(Y_test, rf_pred)
    calculated_acc = accuracy_score(Y_test, rf_pred)
    calculated_pre = precision_score(Y_test, rf_pred)
    calculated_rec = recall_score(Y_test, rf_pred)

    print(f"{calculated_f1 * 100 : .2f}%, \t {reference_f1 * 100 : .2f}% \t | {calculated_acc * 100 : .2f}%, \t {reference_acc * 100 : .2f}% \t | {calculated_pre * 100 : .2f}%, \t {reference_pre * 100 : .2f}% \t | {calculated_rec * 100 : .2f}%, \t {reference_rec * 100 : .2f}%")

With Same Test Data: 
--------------------------------------------------------------------------------------------------------------------------
Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference
F1-score 	 F1-score 	 | Accuracy 	 Accuracy 	 | Precision 	 Precision 	 | Recall 	 Recall
--------------------------------------------------------------------------------------------------------------------------
 100.00%, 	  100.00% 	 |  100.00%, 	  100.00% 	 |  100.00%, 	  100.00% 	 |  100.00%, 	  100.00%


With Different Test Data: 
--------------------------------------------------------------------------------------------------------------------------
Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference 	 | Calculated 	 Reference
F1-score 	 F1-score 	 | Accuracy 	 Accuracy 	 | Precision 	 Precision 	 | Recall 	 Recall
------------------------------------------------------------------------------------------------